In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import librosa
import IPython.display as ipd
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')

In [5]:
import librosa
import os
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from keras import optimizers

In [6]:
emotions_ravdess={
  '01': 1,
  '02': 2,
  '03': 3,
  '04': 4,
  '05': 5,
  '06': 6,
  '07': 7,
  '08': 8
}

# emotions={
#   '01':'neutral',
#   '02':'calm',
#   '03':'happy',
#   '04':'sad',
#   '05':'angry',
#   '06':'fearful',
#   '07':'disgust',
#   '08':'surprised'
# }

emotions_cremad={
  'ANG':5,
  'DIS':7,
  'NEU':1,
  'HAP':3,
  'FEA':6,
  'SAD':4
}

emotions_to_string={
  1:'neutral',
  2:'calm',
  3:'happy',
  4:'sad',
  5:'angry',
  6:'fearful',
  7:'disgust',
  8:'surprised'
}

emotions_intensity = {
  '01':'low',
  '02':'strong',
}

observed_emotions=[1 , 5,  3]

In [7]:
MFCC_NUM = 40
MFCC_MAX_LEN = 2000
SAMPLING_RATE = 44100
feature_dim_1 = MFCC_NUM
feature_dim_2 = MFCC_MAX_LEN
channel = 1

In [8]:
def wav2mfcc(wave, max_len=MFCC_MAX_LEN):
#     mfcc = librosa.feature.mfcc(wave, sr=16000)
    mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [9]:
X = []
y = []

def append_X_Y(label, wave):
    y.append(label)
    mfcc = wav2mfcc(wave)
    X.append(mfcc)

In [10]:
for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\ravdess\\Actor_*\\*.wav"):
    file_name=os.path.basename(file)
    label=emotions_ravdess[file_name.split("-")[2]] 
    if emotions_ravdess[file_name.split("-")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, mono=True, sr=44100)
    wave = wave[::3]
    append_X_Y(label, wave)
    

for file in glob.glob("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\cremad\\*.wav"):
    file_name=os.path.basename(file)
    label=emotions_cremad[file_name.split("_")[2]] 
    if emotions_cremad[file_name.split("_")[2]] not in observed_emotions:
        continue
    wave, sr = librosa.load(file, mono=True, sr=44100)
    wave = wave[::3]
    append_X_Y(label, wave)

In [11]:
X = np.array(X)
y = np.array(y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=True, shuffle=True)

In [13]:
X_train.shape

(2976, 40, 2000)

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(120, (3, 3), activation='relu'))

In [15]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [18]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_split=0.25)

Epoch 1/10
70/70 [==============================] - 56s 795ms/step - loss: 0.7826 - accuracy: 0.6277 - val_loss: 0.7728 - val_accuracy: 0.5995
Epoch 2/10
70/70 [==============================] - 55s 793ms/step - loss: 0.7362 - accuracy: 0.6568 - val_loss: 0.7270 - val_accuracy: 0.6694
Epoch 3/10
70/70 [==============================] - 56s 801ms/step - loss: 0.7478 - accuracy: 0.6528 - val_loss: 0.7933 - val_accuracy: 0.6022
Epoch 4/10
70/70 [==============================] - 81s 1s/step - loss: 0.7101 - accuracy: 0.6720 - val_loss: 0.6961 - val_accuracy: 0.6828
Epoch 5/10
70/70 [==============================] - 75s 1s/step - loss: 0.6853 - accuracy: 0.6980 - val_loss: 0.7311 - val_accuracy: 0.6788
Epoch 6/10
70/70 [==============================] - 62s 888ms/step - loss: 0.6576 - accuracy: 0.7074 - val_loss: 0.7847 - val_accuracy: 0.6586
Epoch 7/10
70/70 [==============================] - 63s 897ms/step - loss: 0.6341 - accuracy: 0.7168 - val_loss: 1.0343 - val_accuracy: 0.5739
Epoch

In [19]:
import pickle

In [21]:
filename = 'emotion_recognition_cnn.pkl'
pickle.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://9c3f1221-258a-476f-84f0-f24faf7f1ba6/assets


In [23]:
model.save("C:\\Users\\User\\Desktop\\diploma\\diploma\\backend\\emotion-recognition\\")

INFO:tensorflow:Assets written to: C:\Users\User\Desktop\diploma\diploma\backend\emotion-recognition\assets


In [41]:
newFile = "C:/Users/User/Desktop/diploma/diploma/backend/speech-diarization/try/17_test.wav"

In [42]:
wave, sr = librosa.load(newFile, mono=True, sr=44100)
wave = wave[::3]
mfcc = wav2mfcc(wave)
X_test = mfcc.reshape(1, feature_dim_1, feature_dim_2, channel)
preds = model.predict(X_test)[0]
classes_x=np.argmax(preds,axis=0)
emotions_to_string[classes_x]

C:\Users\User\AppData\Local\Temp\ipykernel_10876\176945046.py:3: FutureWarning: Pass y=[-0.00201416 -0.00135803 -0.00132751 ... -0.00306702 -0.00126648
  0.00228882] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(wave, n_mfcc=MFCC_NUM, sr=SAMPLING_RATE)


'neutral'